# 1. Import Libraries

In [1]:
import pandas as pd
import datetime
import os
from scipy.stats.stats import pearsonr
import pandas_datareader.data as web
from pandas import Series, DataFrame
from datetime import timedelta

# 2.  Read Dataset

* Since the NDA, the dataset cannot be show below
* In this part, basic dataset info will be check, missing values, statistic value

In [2]:
potoo = pd.read_csv("Uconn Dataset - 8.26.19.csv")
potoo['Date'] = pd.to_datetime(potoo['Date'])

In [3]:
potoo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429737 entries, 0 to 429736
Data columns (total 12 columns):
Date                429737 non-null datetime64[ns]
Ticker              429737 non-null object
Parent Company      429737 non-null object
Brand Name          429737 non-null object
Product Category    429737 non-null object
Product ASIN        429737 non-null object
Product Name        429737 non-null object
Seller Name         429737 non-null object
Stock               429737 non-null int64
Price               429737 non-null float64
Units Sold          429737 non-null int64
Total Sales         429737 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 39.3+ MB


In [4]:
#checing missing values
potoo.isnull().sum()

Date                0
Ticker              0
Parent Company      0
Brand Name          0
Product Category    0
Product ASIN        0
Product Name        0
Seller Name         0
Stock               0
Price               0
Units Sold          0
Total Sales         0
dtype: int64

In [5]:
potoo.describe()

,Stock,Price,Units Sold,Total Sales
count,429737.000000,429737.000000,429737.000000,429737.000000
mean,105.362878,39.290911,7.894622,296.045701
std,165.927872,65.235151,27.430091,1758.678038
min,0.000000,0.990000,1.000000,1.110000
25%,10.000000,12.510000,1.000000,26.410000
50%,37.000000,21.100000,3.000000,62.950000
75%,120.000000,38.090000,6.000000,171.080000
max,997.000000,10000.000000,991.000000,213535.080000


In [6]:
# Change the Coloumn name to make it easier to be used in python
potoo.columns = ["Date", "Ticker", "Parent_Company", 'Brand_Name', 'Product_Category', 'Product_ASIN', 'Product_Name', 'Seller_Name'
                , 'Stock','Price', 'Units_Sold', 'Total_Sales']
#Change the parent company and Brand Name of fitbit to make it clear. The FitBit and Fitbit may cause some confuese in python
potoo.loc[potoo.Parent_Company == 'FitBit', "Parent_Company"] = 'Fitbit'

# 3. Scraped stock price dataset from Yahoo Finance

In [7]:
#Download the stock price dataset from Yahoo Finance
start = datetime.datetime(2017, 4, 12)
end = datetime.datetime(2019, 8, 26)
company = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
Company_Stock = web.DataReader(company, 'yahoo',start = start, end = end)
Company_Stock.drop(Company_Stock.index[-1], inplace = True)
Company_Stock = Company_Stock.stack().reset_index()
Company_Stock.columns = ['Date', 'Ticker', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj_Close']
Company_Stock.tail()

,Date,Ticker,High,Low,Open,Close,Volume,Adj_Close
6253,2019-08-26,NUS,40.599998,39.369999,40.500000,40.139999,493300.0,39.390549
6254,2019-08-26,NWL,15.970000,15.720000,15.880000,15.820000,3045300.0,15.410170
6255,2019-08-26,PLT,29.200001,28.090000,28.590000,29.150000,769600.0,28.984375
6256,2019-08-26,SMG,105.059998,102.750000,103.300003,104.989998,458400.0,104.381058
6257,2019-08-26,YETI,27.799999,25.180000,27.200001,25.780001,1572000.0,25.780001


# 4. Combine Dataset for Further Analysis

In [8]:
#Merge Potoo dataset and Company_Stock Dataset, the stock price is second days` price
Company_Stock['Date'] = pd.to_datetime(Company_Stock['Date'])
Company_Stock['Date'] = Company_Stock['Date'] - timedelta(days=1)
Company_Stock.tail(22)

,Date,Ticker,High,Low,Open,Close,Volume,Adj_Close
6236,2019-08-22,CAJ,25.870001,25.490000,25.650000,25.520000,394200.0,25.520000
6237,2019-08-22,CHD,79.959999,78.139999,79.230003,78.500000,990100.0,78.238220
6238,2019-08-22,FIT,3.215000,3.020000,3.110000,3.030000,4607700.0,3.030000
6239,2019-08-22,GNC,1.900000,1.750000,1.880000,1.800000,820500.0,1.800000
6240,2019-08-22,GPRO,4.210000,3.950000,4.190000,3.960000,3031300.0,3.960000
6241,2019-08-22,HENKY,22.330000,21.969999,22.129999,22.020000,31700.0,22.020000
6242,2019-08-22,NUS,41.700001,39.910000,40.779999,40.110001,629600.0,39.361107
6243,2019-08-22,NWL,16.580000,15.700000,16.530001,15.760000,4343900.0,15.351724
6244,2019-08-22,PLT,28.700001,26.700001,28.410000,28.150000,656900.0,27.990057
6245,2019-08-22,SMG,108.489998,102.720001,108.410004,102.980003,635500.0,101.806084


In [9]:
Company_Stock.to_csv('Company_Stock.csv')

In [10]:
company_stock = pd.read_csv('Company_Stock.csv',index_col=0)
company_stock['Date'] = pd.to_datetime(company_stock['Date'])
potoo_new = pd.merge(potoo, company_stock, on =["Date", "Ticker"], how ='left')
potoo_new.reset_index(drop=True, inplace = True)

# 5. Explorate Data Analysis
* When merging two dataset, we choose the start price of second day since the influence of first day sale will be placed on the stock price of second day
* Carefully clean dataset, some stock of company are not trading for a couple of days, some companies are not list in major market, which is not proper for analysis.
* Drop duplicates
* There is no stock trade in weekend and holiday, so fill the missing value using the stock price in next trade day.

In [11]:
#fill the last day of kinds of company
Ticker = ['NUS', 'CAJ', 'GNC', 'SMG']
for i in Ticker:
    potoo_new.loc[(potoo_new.Ticker ==i) & (potoo_new.Date == datetime.datetime(2019, 8, 23)), ['High', 'Low'
                                                                                           ,'Open', 'Close', 'Volume', 'Adj_Close']] = company_stock.loc[(company_stock.Ticker ==i) & (company_stock.Date == datetime.datetime(2019, 8, 25)), ['High', 'Low'
                                                                                        ,'Open', 'Close', 'Volume', 'Adj_Close']]

In [12]:
potoo_new.loc[(potoo_new.Ticker =='Yeti') & (potoo_new.Date == datetime.datetime(2019, 8, 24)), ['High', 'Low'
                                                                                           ,'Open', 'Close', 'Volume', 'Adj_Close']] = company_stock.loc[(
    company_stock.Ticker == 'Yeti') & (company_stock.Date == datetime.datetime(2019, 8, 25)), ['High', 'Low'
                                                                                        ,'Open', 'Close', 'Volume', 'Adj_Close']]

In [13]:
#explore the dataset -- Missing Value
potoo_new.isnull().sum()

Date                     0
Ticker                   0
Parent_Company           0
Brand_Name               0
Product_Category         0
Product_ASIN             0
Product_Name             0
Seller_Name              0
Stock                    0
Price                    0
Units_Sold               0
Total_Sales              0
High                130211
Low                 130211
Open                130211
Close               130211
Volume              130211
Adj_Close           130211
dtype: int64

In [14]:
#fill missing value by back fill
potoo_new.sort_values(by=['Date','Ticker'],inplace=True)
potoo_new.fillna(method='bfill',inplace=True)
potoo_new.isnull().sum()

Date                0
Ticker              0
Parent_Company      0
Brand_Name          0
Product_Category    0
Product_ASIN        0
Product_Name        0
Seller_Name         0
Stock               0
Price               0
Units_Sold          0
Total_Sales         0
High                0
Low                 0
Open                0
Close               0
Volume              0
Adj_Close           0
dtype: int64

In [15]:
#Delete Duplicates
potoo_new = potoo_new.drop_duplicates()

In [16]:
#choose the second day`s open price as target variable
potoo_new["stock_price"] = potoo_new.Open
potoo_new.drop(["High", "Low", "Open", "Close", "Volume", "Adj_Close"], axis = 1, inplace = True)
df = potoo_new

# 6.Correlation Analysis

* Analyzed the correlation between market attributes and stock price in kinds of segments to find the attribute that places most influence on stock market.
* Set the different time including 1 day, 1 week and 1 month, since the influence may have latency

In [17]:
#Create a correlation dataframe for correlation values
corr = pd.DataFrame(columns=['Ticker','Parent_Company', "Brand_Name",'Product_Category', 'Feature','co_value', 'p_value'])

1 day delay
* the correlation between Tick and stock price

In [18]:
#group by Ticker
f = {
    'Stock':['sum'],
    "Units_Sold" : ['sum'],
   "Price" : ['mean'],
   'Total_Sales': ['sum'],
   'stock_price': ['mean']
}
df_t = df.groupby(['Date', 'Ticker'], as_index = True).agg(f).reset_index()
df_t.columns = ["Date", "Ticker", "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)", "Stock_price"]

In [19]:
#calculate relationship between each variabel and stock price and input in correlation dataframe
company = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
features_list=['Stock(Sum)','Price(mean)','Units_Sold(Sum)','Total_Sales(Sum)']
for i in company:
    for t in features_list:
        a = df_t.loc[df_t.Ticker == i][t]
        b = df_t.loc[df_t.Ticker == i].Stock_price
        c = pearsonr(a,b)
        corr = corr.append({'Ticker': i, 'Feature': t, 'co_value' :c[0], 'p_value': c[1]}, ignore_index=True)

* parent company and stock price 

In [20]:
#Group by Parent Company
df_pc = df.groupby(['Date', 'Ticker', 'Parent_Company'], as_index = True).agg(f).reset_index()
df_pc.columns = ["Date", "Ticker", "Parent_Company", "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)", "Stock_price"]

In [21]:
parent_company = ['Newell Appliances & Cookware', 'Newell Writing', 'GoPro',
       'Newell', 'Newell Outdoor & Recreation', 'Newell Food',
       'Newell Other', 'Newell Connected Home & Security', 'Henkel',
       'Miracle-Gro', 'Nuskin', 'GNC Holdings', 'YETI', 'Fitbit',
       'Plantronics', 'Church and Dwight', 'Canon']
Ticker = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
features_list=[ "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)"]
for p in parent_company:
    for t in features_list:
        a = df_pc.loc[ (df_pc.Parent_Company == p)][t]
        b = df_pc.loc[ (df_pc.Parent_Company == p)].Stock_price
        c = pearsonr(a,b)
        d = df_pc.loc[ (df_pc.Parent_Company == p)]['Ticker'].unique()
        corr = corr.append({'Ticker': d, 'Parent_Company':p,'Feature': t, 'co_value' :c[0], 'p_value': c[1]}, ignore_index=True)

* Brand name and stock price

In [22]:
#Group by Brand Name
df_bn = df.groupby(['Date', 'Ticker','Parent_Company', 'Brand_Name'], as_index = True).agg(f).reset_index()
df_bn.columns = ["Date", "Ticker","Parent_Company", "Brand_Name", "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)", "Stock_price"]

In [23]:
brand_name = ['Sunbeam', 'Waterman', 'GoPro', 'FoodSaver', 'Calphalon',
       'First Alert', 'Contigo', 'Oster', 'Mr. Coffee', 'Ball',
       'Crock-Pot', 'Elmers', 'Rubbermaid', 'Sistema', 'Bicycle',
       'Mr. Sketch', 'Prismacolor', 'X-Acto', 'BRK', 'DYMO', 'Parker',
       'Sharpie', 'Quickie', 'Rubbermaid Commercial Products',
       'Paper Mate', 'WoodWick', 'Coleman', 'Spontex', 'Campingaz',
       'Rotring', 'AeroBed', 'NUK', 'Purex', 'Renuzit', 'Dial', 'Persil',
       'Right Guard', 'Soft Scrub', 'Tone', 'Schwarzkopf', 'Baby Jogger',
       'Schwarzkopf Professional', 'Miracle-Gro', 'Nu Skin', 'Stearns',
       'Schwarzkopf BC REPAIR RESCUE', 'Schwarzkopf Hair Color',
       'Chesapeake Bay Candle', 'GNC', 'YETI', 'Loctite', 'Snuggle',
       'Henkel', 'Combat', 'Pritt', 'Sunlight', 'all', 'Rexona', 'Syoss',
       'Graco', 'bubba', 'Fitbit', 'Plantronics', 'Marmot', 'Mammut',
       'Nair', 'OxiClean', 'Canon', 'Arm & Hammer', 'First Response',
       'Jarrow', 'Phillips', 'Rephresh', 'TROJAN', 'vH essentials', 'EPI',
       'Kaboom', 'Arrid', 'Feline Pine', 'Lil Critters', 'Orajel',
       'Viviscal', 'Waterpik', 'Orange Glo', 'Replens', 'NutraBlast',
       'Fels Naptha', 'Gliss', 'Smooth N Shine', 'Reynolds',
       'Balance Complex']
Ticker = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
features_list=[ "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)"]
for p in brand_name:
    for t in features_list:
        a = df_bn.loc[ (df_bn.Brand_Name == p)][t]
        b = df_bn.loc[ (df_bn.Brand_Name == p)].Stock_price
        c = pearsonr(a,b)
        d = df_bn.loc[ (df_bn.Brand_Name == p)]['Parent_Company'].unique()
        e = df_bn.loc[(df_bn.Brand_Name == p)]['Ticker'].unique()
        corr = corr.append({'Ticker': e, 'Parent_Company':d,'Brand_Name':p,'Feature': t, 'co_value' :c[0], 'p_value': c[1]}, ignore_index=True)

E:\Anaconda\lib\site-packages\scipy\stats\stats.py:3049: RuntimeWarning: invalid value encountered in double_scalars
  0.5*df, 0.5, np.fmin(np.asarray(df / (df + t_squared)), 1.0)
E:\Anaconda\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [24]:
corr.to_csv('correlation_1day_dely.csv')

# 6.2 A week delay

In [25]:
#7day delay
potoo_7day = pd.read_csv("potoo_delay7dayStockPrice.csv")

In [26]:
potoo_7day= potoo_7day.drop_duplicates()

In [27]:
potoo_7day["stock_price"] = potoo_7day.Open
potoo_7day.loc[potoo_7day["Parent Company"] == 'FitBit', "Parent Company"] = 'Fitbit'
potoo_7day.drop(["High", "Low", "Open", "Close", "Volume", "Adj Close"], axis = 1, inplace = True)

In [28]:
df_7 = potoo_7day

In [29]:
f = {
    'Stock':['sum'],
    "Units Sold" : ['sum'],
   "Price" : ['mean'],
   'Total Sales': ['sum'],
   'stock_price': ['mean']
}
df_7_c = df_7.groupby(['Date', 'Ticker'], as_index = True).agg(f).reset_index()
df_7_c.columns = ["Date", "Ticker", "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)", "Stock_price"]

In [30]:
corr_7day= pd.DataFrame(columns=['Ticker','Parent_Company', "Brand_Name",'Product_Category', 'Feature','co_value', 'p_value'])

In [31]:
company = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
features_list=['Stock(Sum)','Price(mean)','Units_Sold(Sum)','Total_Sales(Sum)']
for i in company:
    for t in features_list:
        a = df_7_c.loc[df_7_c.Ticker == i][t]
        b = df_7_c.loc[df_7_c.Ticker == i].Stock_price
        c = pearsonr(a,b)
        corr_7day = corr_7day.append({'Ticker': i, 'Feature': t, 'co_value' :c[0], 'p_value': c[1]}, ignore_index=True)

In [32]:
#group by parent company
df_7_pc = df_7.groupby(['Date', 'Ticker', 'Parent Company'], as_index = True).agg(f).reset_index()
df_7_pc.columns = ["Date", "Ticker", 'Parent_Company', "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)", "Stock_price"]

In [33]:
df_7_pc.Parent_Company.unique()

array(['GoPro', 'Henkel', 'Newell', 'Newell Appliances & Cookware',
       'Newell Food', 'Newell Other', 'Newell Outdoor & Recreation',
       'Newell Writing', 'GNC Holdings', 'Nuskin',
       'Newell Connected Home & Security', 'Miracle-Gro', 'YETI',
       'Fitbit', 'Plantronics', 'Church and Dwight', 'Canon'],
      dtype=object)

In [34]:
parent_company = ['GoPro', 'Henkel', 'Newell', 'Newell Appliances & Cookware',
       'Newell Food', 'Newell Other', 'Newell Outdoor & Recreation',
       'Newell Writing', 'GNC Holdings', 'Nuskin',
       'Newell Connected Home & Security', 'Miracle-Gro', 'YETI',
       'Fitbit', 'Plantronics', 'Church and Dwight', 'Canon']
Ticker = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
features_list=[ "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)"]
for p in parent_company:
    for t in features_list:
        a = df_7_pc.loc[ (df_7_pc.Parent_Company == p)][t]
        b = df_7_pc.loc[ (df_7_pc.Parent_Company == p)].Stock_price
        c = pearsonr(a,b)
        d = df_7_pc.loc[ (df_7_pc.Parent_Company == p)]['Ticker'].unique()
        corr_7day = corr_7day.append({'Ticker': d, 'Parent_Company':p,'Feature': t, 'co_value' :c[0], 'p_value': c[1]}, ignore_index=True)

In [35]:
#group by Brand Name
df_7_bn = df_7.groupby(['Date', 'Ticker','Parent Company', 'Brand Name'], as_index = True).agg(f).reset_index()
df_7_bn.columns = ["Date", "Ticker","Parent_Company", "Brand_Name", "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)", "Stock_price"]

In [36]:
brand_name = ['GoPro', 'Dial', 'Persil', 'Purex', 'Renuzit', 'Right Guard',
       'Soft Scrub', 'BRK', 'Ball', 'Calphalon', 'Contigo', 'Elmers',
       'First Alert', 'FoodSaver', 'Oster', 'Paper Mate', 'Rubbermaid',
       'Rubbermaid Commercial Products', 'Sharpie', 'WoodWick',
       'Mr. Coffee', 'Sunbeam', 'Sistema', 'Bicycle', 'Quickie',
       'Spontex', 'DYMO', 'Mr. Sketch', 'Parker', 'Prismacolor',
       'Rotring', 'Waterman', 'X-Acto', 'GNC', 'Loctite', 'Schwarzkopf',
       'Schwarzkopf Professional', 'Nu Skin', 'NUK', 'AeroBed', 'Coleman',
       'Stearns', 'Miracle-Gro', 'YETI', 'Chesapeake Bay Candle',
       'Campingaz', 'Combat', 'Henkel', 'Pritt', 'Snuggle', 'Sunlight',
       'all', 'Rexona', 'Crock-Pot', 'Tone',
       'Schwarzkopf BC REPAIR RESCUE', 'Syoss', 'Graco', 'Baby Jogger',
       'Fitbit', 'bubba', 'Plantronics', 'Marmot',
       'Schwarzkopf Hair Color', 'Mammut', 'Nair', 'OxiClean', 'Canon',
       'Arm & Hammer', 'First Response', 'Jarrow', 'Phillips', 'Rephresh',
       'TROJAN', 'vH essentials', 'EPI', 'Kaboom', 'Arrid', 'Feline Pine',
       'Lil Critters', 'Orajel', 'Orange Glo', 'Replens', 'Viviscal',
       'Waterpik', 'NutraBlast', 'Fels Naptha', 'Gliss', 'Smooth N Shine',
       'Reynolds', 'Balance Complex']
Ticker = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
features_list=[ "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)"]
for p in brand_name:
    for t in features_list:
        a = df_7_bn.loc[ (df_7_bn.Brand_Name == p)][t]
        b = df_7_bn.loc[ (df_7_bn.Brand_Name == p)].Stock_price
        c = pearsonr(a,b)
        d = df_7_bn.loc[ (df_7_bn.Brand_Name == p)]['Parent_Company'].unique()
        e = df_7_bn.loc[(df_7_bn.Brand_Name == p)]['Ticker'].unique()
        corr_7day = corr_7day.append({'Ticker': e, 'Parent_Company':d,'Brand_Name':p,'Feature': t, 'co_value' :c[0], 'p_value': c[1]}, ignore_index=True)

In [37]:
corr_7day.to_csv('correlation_delay7days.csv')

# 6.3 28 Days

In [38]:
## delay 28days
potoo_28day = pd.read_csv("potoo_delay28dayStockPrice.csv")
potoo_28day = potoo_28day.drop_duplicates()

In [39]:
potoo_28day["stock_price"] = potoo_28day.Open
potoo_28day.loc[potoo_28day["Parent Company"] == 'FitBit', "Parent Company"] = 'Fitbit'
potoo_28day.drop(["High", "Low", "Open", "Close", "Volume", "Adj Close"], axis = 1, inplace = True)
df_28= potoo_28day

In [40]:
#group by TiCKER
f = {
    'Stock':['sum'],
    "Units Sold" : ['sum'],
   "Price" : ['mean'],
   'Total Sales': ['sum'],
   'stock_price': ['mean']
}
df_28_t= df_28.groupby(['Date', 'Ticker'], as_index = True).agg(f).reset_index()
df_28_t.columns = ["Date", "Ticker", "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)", "Stock_price"]

In [41]:
corr_28day= pd.DataFrame(columns=['Ticker','Parent_Company', "Brand_Name",'Product_Category', 'Feature','co_value', 'p_value'])

In [42]:
company = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
features_list=['Stock(Sum)','Price(mean)','Units_Sold(Sum)','Total_Sales(Sum)']
for i in company:
    for t in features_list:
        a = df_28_t.loc[df_28_t.Ticker == i][t]
        b = df_28_t.loc[df_28_t.Ticker == i].Stock_price
        c = pearsonr(a,b)
        corr_28day = corr_28day.append({'Ticker': i, 'Feature': t, 'co_value' :c[0], 'p_value': c[1]}, ignore_index=True)

In [43]:
#group by parent company
df_28_pc = df_28.groupby(['Date', 'Ticker', 'Parent Company'], as_index = True).agg(f).reset_index()
df_28_pc.columns = ["Date", "Ticker", 'Parent_Company', "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)", "Stock_price"]

In [44]:
parent_company = ['GoPro', 'Henkel', 'Newell', 'Newell Appliances & Cookware',
       'Newell Food', 'Newell Other', 'Newell Outdoor & Recreation',
       'Newell Writing', 'GNC Holdings', 'Nuskin',
       'Newell Connected Home & Security', 'Miracle-Gro', 'YETI',
       'Fitbit', 'Plantronics', 'Church and Dwight', 'Canon']
Ticker = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
features_list=[ "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)"]
for p in parent_company:
    for t in features_list:
        a = df_28_pc.loc[ (df_28_pc.Parent_Company == p)][t]
        b = df_28_pc.loc[ (df_28_pc.Parent_Company == p)].Stock_price
        c = pearsonr(a,b)
        d = df_28_pc.loc[ (df_28_pc.Parent_Company == p)]['Ticker'].unique()
        corr_28day = corr_28day.append({'Ticker': d, 'Parent_Company':p,'Feature': t, 'co_value' :c[0], 'p_value': c[1]}, ignore_index=True)

In [45]:
#group by Brand Name
df_28_bn = df_28.groupby(['Date', 'Ticker','Parent Company', 'Brand Name'], as_index = True).agg(f).reset_index()
df_28_bn.columns = ["Date", "Ticker","Parent_Company", "Brand_Name", "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)", "Stock_price"]

In [46]:
brand_name = ['GoPro', 'Dial', 'Persil', 'Purex', 'Renuzit', 'Right Guard',
       'Soft Scrub', 'BRK', 'Ball', 'Calphalon', 'Contigo', 'Elmers',
       'First Alert', 'FoodSaver', 'Oster', 'Paper Mate', 'Rubbermaid',
       'Rubbermaid Commercial Products', 'Sharpie', 'WoodWick',
       'Mr. Coffee', 'Sunbeam', 'Sistema', 'Bicycle', 'Quickie',
       'Spontex', 'DYMO', 'Mr. Sketch', 'Parker', 'Prismacolor',
       'Rotring', 'Waterman', 'X-Acto', 'GNC', 'Loctite', 'Schwarzkopf',
       'Schwarzkopf Professional', 'Nu Skin', 'NUK', 'AeroBed', 'Coleman',
       'Stearns', 'Miracle-Gro', 'YETI', 'Chesapeake Bay Candle',
       'Campingaz', 'Combat', 'Henkel', 'Pritt', 'Snuggle', 'Sunlight',
       'all', 'Rexona', 'Crock-Pot', 'Tone',
       'Schwarzkopf BC REPAIR RESCUE', 'Syoss', 'Graco', 'Baby Jogger',
       'Fitbit', 'bubba', 'Plantronics', 'Marmot',
       'Schwarzkopf Hair Color', 'Mammut', 'Nair', 'OxiClean', 'Canon',
       'Arm & Hammer', 'First Response', 'Jarrow', 'Phillips', 'Rephresh',
       'TROJAN', 'vH essentials', 'EPI', 'Kaboom', 'Arrid', 'Feline Pine',
       'Lil Critters', 'Orajel', 'Orange Glo', 'Replens', 'Viviscal',
       'Waterpik', 'NutraBlast', 'Fels Naptha', 'Gliss', 'Smooth N Shine',
       'Reynolds', 'Balance Complex']
Ticker = ['GPRO', 'FIT', 'NWL', 'PLT', 'HENKY', 'CHD', 'YETI','NUS', 'CAJ', 'GNC', 'SMG']
features_list=[ "Stock(Sum)", "Units_Sold(Sum)", "Price(mean)", "Total_Sales(Sum)"]
for p in brand_name:
    for t in features_list:
        a = df_28_bn.loc[ (df_28_bn.Brand_Name == p)][t]
        b = df_28_bn.loc[ (df_28_bn.Brand_Name == p)].Stock_price
        c = pearsonr(a,b)
        d = df_28_bn.loc[ (df_28_bn.Brand_Name == p)]['Parent_Company'].unique()
        e = df_28_bn.loc[(df_28_bn.Brand_Name == p)]['Ticker'].unique()
        corr_28day = corr_28day.append({'Ticker': e, 'Parent_Company':d,'Brand_Name':p,'Feature': t, 'co_value' :c[0], 'p_value': c[1]}, ignore_index=True)

In [47]:
corr_28day.to_csv("correlation_delay28days.csv")